<a href="https://colab.research.google.com/github/rajarishiR/programm/blob/main/Ensemble_learning_randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [50]:
df = pd.read_csv('/content/train_ctrUa4K.csv')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [51]:
print(df.isnull().sum())
print('.............................................................')
print('shape:',df.shape)
print('.............................................................')
print('data types:',df.dtypes)

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64
.............................................................
shape: (614, 13)
.............................................................
data types: Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object


In [52]:
# droping the null values cause they are samll in num
del df['Loan_ID']
df = df.dropna()
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [53]:
# Encoding the data
from sklearn.preprocessing import LabelEncoder
dfT = pd.get_dummies(df) #encode the data - one hot encoding is done
dfT.columns
dfT = dfT.drop(['Gender_Male','Married_No','Education_Graduate','Self_Employed_No'], axis = 1)
dfT['Dependents'] = dfT['Dependents_0'] + (2 * dfT['Dependents_1']) + (3 * dfT['Dependents_2']) + (4 * dfT['Dependents_3+']) - 1
dfT = dfT.drop(['Dependents_0','Dependents_1','Dependents_2','Dependents_3+'], axis = 1)
dfT['Loan_Status'] = dfT['Loan_Status_Y']
dfT = dfT.drop(['Loan_Status_Y','Loan_Status_N'], axis = 1)
dfT.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Dependents,Loan_Status
1,4583,1508.0,128.0,360.0,1.0,0,1,0,0,1,0,0,1,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,0,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,1,0,0,0,1,0,1
4,6000,0.0,141.0,360.0,1.0,0,0,0,0,0,0,1,0,1
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,0,1,2,1


In [54]:
x = dfT[dfT.columns[:-1]].values
y = dfT['Loan_Status'].values
# Train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25)

In [55]:
# we will cause one of our algorithm is going to be knn
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
x_train = std.fit_transform(x_train)
x_test = std.transform(x_test)

In [56]:
#Voting concept
from sklearn.ensemble import VotingClassifier # this is the function that ensembles my model
from sklearn.linear_model import LogisticRegression # logistic regression model
from sklearn import tree # Decision tree model
from sklearn.neighbors import KNeighborsClassifier # Knn model
from sklearn.metrics import roc_auc_score # auroc value to get the good evaluation metric
model1 = LogisticRegression(random_state=1)
model2 = tree.DecisionTreeClassifier(max_depth = 2, random_state=2)
model3 = KNeighborsClassifier(3)
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2),('knn',model3)], voting='soft') # combining all the models 
model.fit(x_train,y_train)
pred = model.predict(x_test)
# auroc score to know how good my model is
roc_auc_score(y_test,model.predict_proba(x_test)[:,1])

0.767971758664955

In [57]:
#Bagging concept
from sklearn.ensemble import BaggingClassifier
from sklearn import tree # we are using bagging concept on the decision tree 
mod = BaggingClassifier(tree.DecisionTreeClassifier(max_depth= 1, random_state = 1), n_estimators=100) # estimatore are 
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.8083333333333333

# Random Forest

In [58]:
import pandas as pd
import numpy as np


In [59]:
df = pd.read_csv('/content/temps.csv')
df.head()

,year,month,day,week,temp_2,temp_1,average,actual,friend
0,2016,1,1,Fri,45,45,45.6,45,29
1,2016,1,2,Sat,44,45,45.7,44,61
2,2016,1,3,Sun,45,44,45.8,41,56
3,2016,1,4,Mon,44,41,45.9,40,53
4,2016,1,5,Tues,41,40,46.0,44,41


In [60]:
# The random forest dosent care the type of encoding
df.week = df.week.map({'Sun':0,'Mon':1,'Tues':2,'Wed':3,'Thurs':4,'Fri':5,'Sat':6})
a = df.pop('actual')
df.insert(8,'actual',a)
df.head()

,year,month,day,week,temp_2,temp_1,average,friend,actual
0,2016,1,1,5,45,45,45.6,29,45
1,2016,1,2,6,44,45,45.7,61,44
2,2016,1,3,0,45,44,45.8,56,41
3,2016,1,4,1,44,41,45.9,53,40
4,2016,1,5,2,41,40,46.0,41,44


In [61]:
x = df[df.columns[:-1]].values
y = df.actual.values

In [62]:
# train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25)

In [63]:
# importing random forest
from sklearn.ensemble import RandomForestRegressor # using regressor cause its an regression problem
from sklearn.metrics import r2_score
rf = RandomForestRegressor(n_estimators= 100, max_depth = 3, max_features='sqrt') # instaling the model with max depth and max feature (its the square root of the feature)
# now training the model with training data
rf.fit(x_train,y_train)

RandomForestRegressor(max_depth=3, max_features='sqrt')

In [64]:
#predicting with test data
predictions = rf.predict(x_test)

In [65]:
# calculating absolute error and mean squared error
# absolute error
errors = abs(predictions - y_test)
# mean squared error
print('Mean squared value :',np.mean(errors))

Mean squared value : 4.800794915046533


In [66]:
# r squared value
r2_score(predictions,y_test)

0.61163465649016

In [67]:
X = df.columns[:-1].values

In [68]:
# Getting feature importance
# its a universal code can be used for most of the decision tree important features
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(X, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];


Variable: temp_1               Importance: 0.29
Variable: average              Importance: 0.28
Variable: temp_2               Importance: 0.27
Variable: month                Importance: 0.1
Variable: friend               Importance: 0.05
Variable: day                  Importance: 0.01
Variable: week                 Importance: 0.01
Variable: year                 Importance: 0.0
